# Setup

In [1]:
%pip install spuco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.1 MB/s eta 0:00:00
  Created wheel for grad-cam: filename=grad_cam-1.5.4-py3-none-any.whl size=39588 sha256=d3524350268b51743c4bebf8fb52106f9a3fd31addbf5eec5bee0562bbf8a63f
  Stored in directory: /root/.cache/pip/wheels/50/b0/82/1f97b5348c7fe9f0ce0ba18497202cafa5dec4562bd5292680
Successfully built grad-cam


In [2]:
import torch

device = torch.device("cpu")

In [3]:
from spuco.utils import set_seed

set_seed(0)

# Dataset Creation

In [4]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train",
    label_noise=0.001
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 54.2MB/s]


Extracting /data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 2.55MB/s]

Extracting /data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 14.5MB/s]


Extracting /data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 3.41MB/s]

Extracting /data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



In [5]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]

valset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()


In [6]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [7]:
from spuco.models import model_factory
from spuco.robust_train import ERM
from torch.optim import SGD
import numpy as np
from spuco.robust_train import CustomSampleERM
from spuco.utils import Trainer
from spuco.group_inference import JTTInference
from spuco.robust_train import CustomSampleERM
from spuco.evaluate import Evaluator
import random
random.seed(10)

# Train using JTT

In [8]:
#shuffle training set
print('Total trainset size: ' + str(len(trainset)))
shuffled_indices = np.arange(0, len(trainset))
random.shuffle(shuffled_indices)

Total trainset size: 48004


## Group Inference

In [9]:
def gi_subset(num_partitions, partition_id, trainset, device, shuffled_indices, verbose=False):
  #generate subset index
  partition_len = int(len(trainset)/num_partitions)
  train_index = shuffled_indices[partition_len*partition_id:partition_len*partition_id+partition_len]
  train_index = train_index.tolist()
  #print(train_index)
  #train on subset
  infer_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
  gi_trainer = CustomSampleERM(
      model=infer_model,
      trainset=trainset,
      batch_size=64,
      optimizer=SGD(infer_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
      num_epochs=2,
      indices=train_index,
      device=device,
      verbose=verbose
  )
  gi_trainer.train()
  #get labels
  gi_trainer_results = Trainer(
    model=infer_model,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(infer_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=verbose
  )
  output = gi_trainer_results.get_trainset_outputs()
  max_output = torch.max(output, dim=1)
  pred_labels = max_output.indices
  pred_labels = pred_labels.tolist()
  actual_labels = trainset.labels
  #jtt inference on subset
  jtt = JTTInference([pred_labels[i] for i in train_index], [actual_labels[i] for i in train_index])
  gi = jtt.infer_groups()
  nominal_upsample = len(gi[(0, 0)])/len(gi[(0, 1)])
  if verbose:
    print('\nNominal Upsample: ' + str(nominal_upsample))
    print('Majority class size: ' + str(len(gi[(0, 0)])))
    print('Minority class size: ' + str(len(gi[(0, 1)])))
  #return majority, minority group indices
  majority = [train_index[i] for i in gi[(0, 0)]]
  minority = [train_index[i] for i in gi[(0, 1)]]
  return majority, minority

In [10]:
def full_gi(num_partitions, trainset, device, shuffled_indices, verbose=False):
  maj, min = gi_subset(num_partitions, 0, trainset, device, shuffled_indices, verbose)
  for i in range(1, num_partitions):
    maj_temp, min_temp = gi_subset(num_partitions, i, trainset, device, shuffled_indices, verbose)
    maj.extend(maj_temp)
    min.extend(min_temp)
  print('Majority class size: ' + str(len(maj)))
  print('Minority class size: ' + str(len(min)))
  nominal_upsample = int(0.8*len(maj)/len(min))
  upsampled_indices = [i for i in min for x in range(nominal_upsample)]
  upsampled_indices.extend(maj)
  print('Upsampled length: ' + str(len(upsampled_indices)))
  return upsampled_indices

In [11]:
#upsampled_indices = full_gi(2, trainset, device, shuffled_indices, verbose=True)

## Robust Training

In [12]:
def robust_train(trainset, testset, device, upsampled_indices, verbose=False):
  #robust training
  robust_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
  rt_erm = CustomSampleERM(
      model=robust_model,
      trainset=trainset,
      batch_size=64,
      optimizer=SGD(robust_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
      num_epochs=3,
      indices=upsampled_indices,
      device=device,
      verbose=verbose
  )
  rt_erm.train()
  #evaluate robust model
  rt_evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=robust_model,
    device=device,
    verbose=verbose
  )
  rt_evaluator.evaluate()
  wg_acc = rt_evaluator.worst_group_accuracy
  avg_acc = rt_evaluator.average_accuracy
  spur_acc = rt_evaluator.evaluate_spurious_attribute_prediction()
  print('\nWorst group accuracy: ' + str(wg_acc))
  print('Average accuracy: ' + str(avg_acc))
  print('Spurious attribute accuracy: ' + str(spur_acc))
  return wg_acc, avg_acc, spur_acc

In [13]:
#robust_train(trainset, testset, device, upsampled_indices)

## Sweep Num Partitions

In [14]:
shuffled_indices = np.arange(0, len(trainset))
upsampled_indices = full_gi(16, trainset, device, shuffled_indices, verbose=True)
wg, avg, spur = robust_train(trainset, testset, device, upsampled_indices)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 0:  13%|█▎        | 6/47 [00:00<00:03, 12.80batch/s, accuracy=26.5625%, loss=1.61]

 | Epoch 0 | Loss: 1.612053632736206 | Accuracy: 26.5625%
 | Epoch 0 | Loss: 1.6142735481262207 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.6233304738998413 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.6160612106323242 | Accuracy: 14.0625%
 | Epoch 0 | Loss: 1.6120413541793823 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6202679872512817 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6101903915405273 | Accuracy: 39.0625%
 | Epoch 0 | Loss: 1.608681321144104 | Accuracy: 32.8125%
 | Epoch 0 | Loss: 1.6124258041381836 | Accuracy: 26.5625%


Epoch 0:  30%|██▉       | 14/47 [00:00<00:01, 23.69batch/s, accuracy=26.5625%, loss=1.59]

 | Epoch 0 | Loss: 1.6107014417648315 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.605733036994934 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6114810705184937 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.614184856414795 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.602052092552185 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6059197187423706 | Accuracy: 10.9375%
 | Epoch 0 | Loss: 1.6009836196899414 | Accuracy: 14.0625%
 | Epoch 0 | Loss: 1.5916320085525513 | Accuracy: 26.5625%


Epoch 0:  49%|████▉     | 23/47 [00:01<00:00, 32.10batch/s, accuracy=46.875%, loss=1.58]

...

 | Epoch 1 | Loss: 0.024093404412269592 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.006720324978232384 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.005148702301084995 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.007735694758594036 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.005942304618656635 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.006414698902517557 | Accuracy: 100.0%


Getting model outputs: 100%|██████████| 751/751 [00:13<00:00, 54.90batch/s]


Nominal Upsample: 175.47058823529412
Majority class size: 2983
Minority class size: 17
Majority class size: 47696
Minority class size: 304
Upsampled length: 85696



Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:10<00:00,  2.48it/s]



Worst group accuracy: ((4, 1), 14.105793450881611)
Average accuracy: 99.18194623603048
Spurious attribute accuracy: 19.64
